# Flags and Performance

In earlier example we have occasionally used flags that modified the behavior of regular expressions. In this lesson we look at them more systematically.  

Another issue that you may encounter is poor performance of regular expressions against long string.  The examples in these lessons are small, but in principle you may use (complex) regular expressions against strings that are tens or hundreds of megabytes in size.

In [ ]:
from src.setup import *

## Catastrophic Backtracking

The engine underneath a regular expression module can make a large asymptotic difference in performance.  The story here is complicated and involves a lot of computer science and math; moreover, providing better bounds on "pathological" cases sometimes makes worse typical behavior on "normal" cases.  The documentation of the standard library `re` module notes the third party `regex` module which both has enhanced functionality (some more experimental) and uses a different underlying algorithmic that handles the "monsters" vastly better.

A contrived example below will show how rephrasing a regular expression can make a large difference.  We construct two strings of the letter 'a' terminated by either 'Y' or 'Z'.  The tricky part here is that different subgroups match overlapping sets of patterns.  That in itself is perfectly reasonable and not uncommon

In [2]:
success = 'a'*30 + 'Z'
failure = 'a'*30 + 'Y'

We might first try to match "a string of lowercase letters followed by 'Z'".  This works fine for both the success case and where it fails at the last character.

In [3]:
pat1 = re.compile(r'[a-z]+Z')

In [4]:
%%time
re.match(pat1, success) or "No Match!"

CPU times: user 10 µs, sys: 1 µs, total: 11 µs
Wall time: 14.1 µs


<re.Match object; span=(0, 31), match='aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaZ'>

In [5]:
%%time
re.match(pat1, failure) or "No Match!"

CPU times: user 12 µs, sys: 1 µs, total: 13 µs
Wall time: 16.7 µs


'No Match!'

For a next variation, we want to match:

* a string of (at least one) vowels
* followed one of more of the first five letters of alphabet
* last two subpatterns occurring at least once together
* ending with a 'Z'

This has a similar feel to the part codes we looked at that had 2-4 letters followed by 1-3 digits; identifying a list of only part codes would have similar logic to this.

In [6]:
pat2 = re.compile(r'([aeiou]+[abcde]+)+Z')

In [7]:
%%time
re.match(pat2, success)

CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 12.2 µs


<re.Match object; span=(0, 31), match='aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaZ'>

In [8]:
%%time
re.match(pat2, failure) or "No Match!"

CPU times: user 38.7 s, sys: 67.2 ms, total: 38.8 s
Wall time: 38.6 s


'No Match!'

## What Went Wrong?

We started at couple tens of microseconds for either the success or failure case.  For the success case, the time is similar.  But for the failure case we are into tens of **seconds**, i.e. a million times worse.  It gets worse than that though: the complexity is exponential; in other words, the worst case gets worse at a rate of $O(2^N)$ (where N is the length of the target string).  For example, the analagous pattern with 35 'a's takes 20 minutes, and failing to match ten thousand 'a's would take longer than the lifespan of the universe.

The problem is that the regular expression (depending on engine) needs to consider every possible combination of adjacent subpattern matches, and every number of occurrence counts of those, before it can conclude that there is no 'Z' at the end of the initial group.

We can fix this is several different ways, but finding one for your particular problem can be something of a dark art.  In the example, we might lookahead for something much more general to get to failures quickly.  Anything that could match would need to be a more than one generic alphanumeric character followed by a 'Z'.  This allows the targets with no 'Z' characters at all to fail immediately without recursive backtracking.

In [9]:
pat3 = re.compile(r'(?=\w+Z)([aeiou]+[abcde]+)+Z')

In [10]:
%%time
re.match(pat3, success) or "No Match!"

CPU times: user 14 µs, sys: 0 ns, total: 14 µs
Wall time: 17.9 µs


<re.Match object; span=(0, 31), match='aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaZ'>

In [11]:
%%time
re.match(pat3, failure) or "No Match!"

CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 13.1 µs


'No Match!'

Let us see that the improved pattern works even on vastly larger target strings.  A million character string moves us to milliseconds, but not into millennia.

In [12]:
success2 = 'a'*1_000_000 + 'Z'
failure2 = 'a'*1_000_000 + 'Y'

In [13]:
%%time
re.match(pat3, success2) or "No Match!"

CPU times: user 16.4 ms, sys: 1 µs, total: 16.4 ms
Wall time: 16.9 ms


<re.Match object; span=(0, 1000001), match='aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa>

In [14]:
%%time
re.match(pat3, failure2) or "No Match!"

CPU times: user 8.7 ms, sys: 3 µs, total: 8.71 ms
Wall time: 8.72 ms


'No Match!'

## Regular Expression Flags

There are six flags you may use to change the meaning of patterns.  When you use these, you pass them using the keyword argument `flags` of `re.compile()` and of other `re` module functions.  However, the `flags` argument only works if a string rather than compiled pattern is passed to other functions.

Flag, Short   | Description
--------------|------------------------------------------------------------------------
ASCII, A      | `\w`, `\b`, `\s` and `\d` match only on ASCII characters
DOTALL, S     | `.` matches any character, including newlines
IGNORECASE, I | Case-insensitive matches
LOCALE, L     | Locale-aware match
MULTILINE, M  | `^` and `$` match each line, not whole string
VERBOSE, X    | Enable verbose regular expressions

Flags may be combined by bitwise or'ing them.  Matching per-line, case-insensitively, first words of lines that start with latter part of alphabet:

In [16]:
show(r'^[l-z][a-z]*', rhyme, flags=re.M | re.I)

An additional cryptic means of choosing flags is available.  They can be part of the pattern itself using a prefix to the whole pattern or a group.  The inline pattern codes are any of `aiLmsux`, with `aLu` permitted per individual group as well.  The `u` here means to use Unicode (in contrast to ASCII or LOCALE).

In [19]:
# Entire pattern is case-insensitive and multiline
# The first-letter group is interpreted as ASCII
re.findall(r'(?im)(?a:^[l-z])[a-z]*', rhyme)

['Mary', 'went', 'to']